# Problem 3: Data cleaning with Pandas

_Version 1.8_

**Pro-tips.** If your program behavior seem strange, try resetting the kernel and rerunning everything. If you mess up this notebook or just want to start from scratch, save copies of all your partial responses and use `Actions` $\rightarrow$ `Reset Assignment` to get a fresh, original copy of this notebook. (_Resetting will wipe out any answers you've written so far, so be sure to stash those somewhere safe if you intend to keep or reuse them!_)

Often (nearly always) the data we are working with will need some type of cleaning or rearranging before it can be used. In this problem, you will be working with some messy data. The dataset that we will be working with is called "sales.csv". It contains some ticket sale data, accessed from [DataCamp's R Tutorial](https://www.datacamp.com/courses/importing-cleaning-data-in-r-case-studies). You will be completing this problem with Pandas. 

Our goal will be to take this messy data set, and after doing some data cleaning, we will determine which 5 cities have the highest ticket prices on average, after accounting for inflation. The excercises will step you through this process. 

**Exercise 0** (ungraded). Before we can begin, let's import the dataset, called `sales.csv`, into a pandas dataframe. We will alias it as `data`. Use `data.head()` to explore the data.

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("sales.csv", encoding='latin1')
data.head(5)

,Unnamed: 0,event_id,primary_act_id,secondary_act_id,purch_party_lkup_id,event_name,primary_act_name,secondary_act_name,major_cat_name,minor_cat_name,...,edu_1st_indv_val,edu_2nd_indv_val,adults_in_hh_num,married_ind,child_present_ind,home_owner_ind,occpn_val,occpn_1st_val,occpn_2nd_val,dist_to_ven
0,1,abcaf1adb99a935fc661,43f0436b905bfa7c2eec,b85143bf51323b72e53c,7dfa56dd7d5956b17587,Xfinity Center Mansfield Premier Parking: Flor...,XFINITY Center Mansfield Premier Parking,NaN,MISC,PARKING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,6c56d7f08c95f2aa453c,1a3e9aecd0617706a794,f53529c5679ea6ca5a48,4f9e6fc637eaf7b736c2,Gorge Camping - dave matthews band - sept 3-7,Gorge Camping,Dave Matthews Band,MISC,CAMPING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0
2,3,c7ab4524a121f9d687d2,4b677c3f5bec71eec8d1,b85143bf51323b72e53c,6c2545703bd527a7144d,Dodge Theatre Adams Street Parking - benise,Parking Event,NaN,MISC,PARKING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,394cb493f893be9b9ed1,b1ccea01ad6ef8522796,b85143bf51323b72e53c,527d6b1eaffc69ddd882,Gexa Energy Pavilion Vip Parking : kid rock wi...,Gexa Energy Pavilion VIP Parking,NaN,MISC,PARKING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,55b5f67e618557929f48,91c03a34b562436efa3c,b85143bf51323b72e53c,8bd62c394a35213bdf52,Premier Parking - motley crue,White River Amphitheatre Premier Parking,NaN,MISC,PARKING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
assert data.shape[0] == (5000),"Number of rows is incorrect"
assert data.shape[1] == (46),"Number of collumns is incorrect"

**Exercise 1** (2 points). We have some null values and a lot of unnecessary columns in our `data`. We should drop the rows that have a null value in `onsale_dt` and only keep the columns that we need for this task: `event_disp_name, tickets_purchased_qty, trans_face_val_amt, onsale_dt, venue_city, and venue_state.` 

Let's alias this dataframe as `data_clean`. Please makes sure your indices are preserved. Do not change the name of your data frame going forward or the autograder won't work. 

Using `inplace=True` will be useful for you to do this. You need to be careful when using this, however. Because it modifies the current dataframe you can run into errors that you may not expect. For example, if you drop a column, that column will no longer exist in your dataframe. Trying to rerun the cell will cause an error because you will be trying to drop a column that doesn't exist. 

Just make sure your final data frame in each section is called `data_clean`.

In [3]:
data_clean = data[['event_disp_name', 'tickets_purchased_qty', 'trans_face_val_amt', 'onsale_dt', 'venue_city', 'venue_state']]
data_clean = data_clean[data_clean['onsale_dt'].notnull()]



In [4]:
# Test: Exercise1

assert data_clean.shape == (4899,6), "Your dataframe is not the right shape"
assert data_clean.shape[0] == (4899), "Number of rows is wrong"
assert data_clean.shape[1] == (6), "Number of columns is wrong"
assert data_clean.index.max() == 4999, "Your index is wrong"
assert data_clean.index.min() == 0, "Your index is wrong"
print("\n(Passed!)")



(Passed!)


**Exercise 2** (4 points total -- 1 point "exposed" and 3 points hidden).

Now that we have the data down to the 6 columns we care about, we can continue.

You'll notice that we do not yet have a per ticket price. So that we are comparing apples to apples, we must  divide the face value amount by the quantity of tickets purchased, then increase this by the appropriate amount of inflation based on the year the tickets were sold to the general public (`onsale_dt`).  Alias this as `per_ticket_price`. Make sure to round it to two decimal places using the built in round function (round()), since we are working with dollar values. Only round at the end.

We are provided the [inflation rates](https://www.thebalance.com/u-s-inflation-rate-history-by-year-and-forecast-3306093) below for each respective year in the dataset. Increase the `per_ticket_price` by this amount, depending on the year the tickets were sold to the general public (`onsale_dt`). You will need to pull the year from `onsale_dt` to do this. When you do, alias it as 'year' and store the values as strings. Your dataframe should now have two new columns that are labeled `'per_ticket_price'` and `'year'`. There is no need to multiply inflation rates together. Just take each year's inflation rate as independent in your calculations. 

Additionally, now please reset your index after doing the calculations.

| 2004 | 2005 | 2006 | 2007 | 2008 | 2009 | 2010 | 2011 | 2012 | 2013 | 2014 | 2015 | 2016 |
|------|------|------|------|------|------|------|------|------|------|------|------|------|
| 3.3% | 3.4% | 2.5% | 4.1% | 0.1% | 2.7% | 1.5% | 3.0% | 1.7% | 1.5% | 0.8% | 0.7% | 2.1% |

In [5]:
data_clean['onsale_dt'] = pd.to_datetime(data_clean['onsale_dt'])
data_clean['year'] = data_clean['onsale_dt'].dt.year
data_clean['year'] = data_clean['year'].apply(str)

conditions = [
    (data_clean['year'] == '2004'),
    (data_clean['year'] == '2005'),
    (data_clean['year'] == '2006'),
    (data_clean['year'] == '2007'),
    (data_clean['year'] == '2008'),
    (data_clean['year'] == '2009'),
    (data_clean['year'] == '2010'),
    (data_clean['year'] == '2011'),
    (data_clean['year'] == '2012'),
    (data_clean['year'] == '2013'),
    (data_clean['year'] == '2014'),
    (data_clean['year'] == '2015'),
    (data_clean['year'] == '2016')
    ]
choices = [
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.033)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.034)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.025)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.041)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.001)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.027)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.015)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.030)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.017)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.015)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.008)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.007)),
    (data_clean['trans_face_val_amt']/data_clean['tickets_purchased_qty'] * (1.021))
    ]
data_clean['per_ticket_price'] = np.select(conditions, choices, default=0)
data_clean['per_ticket_price'] = data_clean['per_ticket_price'].round(2)
data_clean = data_clean.reset_index(drop=True)
data_clean = data_clean[['event_disp_name', 'tickets_purchased_qty', 'trans_face_val_amt', 'onsale_dt', 'venue_city', 'venue_state','year','per_ticket_price']]
data_clean


,event_disp_name,tickets_purchased_qty,trans_face_val_amt,onsale_dt,venue_city,venue_state,year,per_ticket_price
0,Xfinity Center Mansfield Premier Parking: Flor...,1,45.0,2015-05-15,MANSFIELD,MASSACHUSETTS,2015,45.32
1,Gorge Camping - dave matthews band - sept 3-7,1,75.0,2009-03-13,QUINCY,WASHINGTON,2009,77.02
2,Dodge Theatre Adams Street Parking - benise,1,5.0,2006-02-25,PHOENIX,ARIZONA,2006,5.12
3,Gexa Energy Pavilion Vip Parking : kid rock wi...,1,20.0,2011-04-22,DALLAS,TEXAS,2011,20.60
4,Premier Parking - motley crue,1,20.0,2005-03-04,AUBURN,WASHINGTON,2005,20.68
5,Fast Lane Access: Journey,2,10.0,2012-04-11,SAN BERNARDINO,CALIFORNIA,2012,5.08
6,Phil Lesh & friends-pnc bank arts center holmd...,1,30.0,2006-05-12,HOLMDEL,NEW JERSEY,2006,30.75
7,Star Parking Presented By Hertz: Train - whit...,1,28.0,2013-02-16,AUBURN,WASHINGTON,2013,28.42
8,Brad Paisley: Hertz Premier Parking,1,20.0,2012-08-18,PHOENIX,ARIZONA,2012,20.34
9,Shlohmo,1,25.0,2014-12-12,SAN FRANCISCO,CALIFORNIA,2014,25.20


In [6]:
#Test Ex 2
assert(len(data_clean['year']) == 4899), 'Your dataset is the wrong length'
index = np.random.choice(len(data_clean['year']))
assert type(data_clean['year'][index]) == str, 'Years are wrong type'
assert data_clean.shape == (4899,8), "Your dataframe is not the correct shape"
assert abs(data_clean['per_ticket_price'][0] - 45.31) < 0.02
assert abs(data_clean['per_ticket_price'][1] - 77.02) < 0.02
assert abs(data_clean['per_ticket_price'][2] - 5.12) < 0.02



print("\n(Passed!)")


(Passed!)


In [7]:
# Test: Exercise2
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**Exercise 3** (1 point). The fields `venue_city` and `venue_state` are unfortunately not in the format that we need. Because we will later be asking for the cities with the highest ticket prices, we should combine `venue_city` and `venue_state` in a City, State format (ex. `Atlanta, Georgia`). Doing so will avoid confusion and errors related to cities that are in more than one state ([Springfield](https://www.worldatlas.com/articles/most-common-town-and-city-names-in-the-u-s-a.html) or [Gainesville](https://en.wikipedia.org/wiki/Gainesville) for example). Save this new column as `location`, and drop `venue_city` and `venue_state`. 

In [8]:
data_clean['location'] = data_clean['venue_city'].map(str) + ', ' + data_clean['venue_state']
data_clean = data_clean[['event_disp_name', 'tickets_purchased_qty', 'trans_face_val_amt', 'onsale_dt', 'year','per_ticket_price','location']]
data_clean.head()

,event_disp_name,tickets_purchased_qty,trans_face_val_amt,onsale_dt,year,per_ticket_price,location
0,Xfinity Center Mansfield Premier Parking: Flor...,1,45.0,2015-05-15,2015,45.32,"MANSFIELD, MASSACHUSETTS"
1,Gorge Camping - dave matthews band - sept 3-7,1,75.0,2009-03-13,2009,77.02,"QUINCY, WASHINGTON"
2,Dodge Theatre Adams Street Parking - benise,1,5.0,2006-02-25,2006,5.12,"PHOENIX, ARIZONA"
3,Gexa Energy Pavilion Vip Parking : kid rock wi...,1,20.0,2011-04-22,2011,20.60,"DALLAS, TEXAS"
4,Premier Parking - motley crue,1,20.0,2005-03-04,2005,20.68,"AUBURN, WASHINGTON"


In [9]:
# Test: Exercise3 

assert 'location' in data_clean.columns, "Did you add location?"
assert 'venue_city' not in data_clean.columns, "Did you drop venue_city?"
assert 'venue_state' not in data_clean.columns, "Did you drop venue_state?"
assert data_clean['location'][0] == 'MANSFIELD, MASSACHUSETTS'
print("\n(Passed!)")


(Passed!)


**Exercise 4** (2 points). 
Finally, let's find the average ticket price per city. List only the `location` and `per_ticket_price` for the **top 5 cities.** Call this dataframe `highest_cities` and please make sure its index is 0, 1, 2, 3, 4 (*hint the index can be accomplished with just one function call*). Remember to round your price to two decimal places using round().

In the cell below you can find both a sample solution and a sample dataset for you to try out. 

In [10]:
sample_data = pd.DataFrame()
sample_data['per_ticket_price'] = [4.80, 15.16, 23.42]
sample_data['location'] = ['ATLANTA, GEORGIA', 'LOS ANGELES, CALIFORNIA', 'ATLANTA, GEORGIA']

sample_sol = pd.DataFrame()
sample_sol['location'] = ['LOS ANGELES, CALIFORNIA', 'ATLANTA, GEORGIA']
sample_sol['per_ticket_price'] = [15.16, 14.11]

sample_sol

,location,per_ticket_price
0,"LOS ANGELES, CALIFORNIA",15.16
1,"ATLANTA, GEORGIA",14.11


In [11]:
highest_cities = data_clean.groupby(['location'])['per_ticket_price'].mean().reset_index()
highest_cities = highest_cities[['location', 'per_ticket_price']].sort_values(by='per_ticket_price',ascending=False)
highest_cities = highest_cities.reset_index(drop=True)
highest_cities['per_ticket_price'] = highest_cities['per_ticket_price'].round(2)
highest_cities = highest_cities.nlargest(5,'per_ticket_price')
highest_cities

,event_disp_name,tickets_purchased_qty,trans_face_val_amt,onsale_dt,year,per_ticket_price,location
0,Xfinity Center Mansfield Premier Parking: Flor...,1,45.0,2015-05-15,2015,45.32,"MANSFIELD, MASSACHUSETTS"
1,Gorge Camping - dave matthews band - sept 3-7,1,75.0,2009-03-13,2009,77.02,"QUINCY, WASHINGTON"
2,Dodge Theatre Adams Street Parking - benise,1,5.0,2006-02-25,2006,5.12,"PHOENIX, ARIZONA"
3,Gexa Energy Pavilion Vip Parking : kid rock wi...,1,20.0,2011-04-22,2011,20.60,"DALLAS, TEXAS"
4,Premier Parking - motley crue,1,20.0,2005-03-04,2005,20.68,"AUBURN, WASHINGTON"


In [14]:
highest_cities

,location,per_ticket_price
0,"READING, PENNSYLVANIA",196.36
1,"FLUSHING, NEW YORK",190.59
2,"PITTSBURGH, PENNSYLVANIA",156.46
3,"PASADENA, CALIFORNIA",145.59
4,"FOXBOROUGH, MASSACHUSETTS",145.57


In [15]:
# Test: Exercise4
assert(highest_cities.iloc[0][0] == 'READING, PENNSYLVANIA')

In [16]:
# Test: Exercise4_hidden

###
### AUTOGRADER TEST - DO NOT REMOVE
###


**Fin!** You’ve reached the end of this part. Don’t forget to restart and run all cells again to make sure it’s all working when run in sequence; and make sure your work passes the submission process. Good luck!